# Basis pursuit with a single "Gaussian" observation

Let $y\in\mathbb{R}, \sigma\in\mathbb{R}_+^*$ and consider 

$$
\underset{u\in L^2(\mathbb{R}^2)}{\text{inf}} ~ |Du|(\mathbb{R}^2) \text{ s.t. } 
\int_{\mathbb{R}^2}\text{exp}(-\frac{||x||^2}{2\sigma^2}) ~ u(x) ~ dx = y
$$

One can show that there exists a solution of the form $u_{\alpha,R}=\alpha\,\mathbb{1}_{B(0, R)}$

Now one can also show that $u_{\alpha,R}$ is a solution if and only if $R=\sigma$ and 
$\alpha=y \, / \, [2\pi\sigma^2(1-\text{exp}(-1/2))]$

Let us investigate whether we are able to recover this result numerically or not ...

In [1]:
import numpy as np

from math import pi, exp

from tvsfw import GaussianPolynomial

In [2]:
sigma = 1.0
y = 2 * pi * sigma ** 2 * (1 - exp(-1/2))